In [ ]:
!cp nasze_dane.ipynb /media/raid/shelfwise/projects/notebooks/kkol/

In [ ]:
# !pip install tensorflow-gpu==1.14.0

In [ ]:
%load_ext autoreload
%autoreload 2

from classification_models.tfkeras import Classifiers
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

tf.enable_eager_execution()

keras = tf.keras
K = tf.keras.backend

In [ ]:
import iic_datasets_ops
import iic_loss_ops
import utils
import models
import mnist_ops

In [ ]:

batch_size = 64
num_classes = 10
num_repeats = 2
input_shape = [128, 128, 3]

# dataset

train_dataset = # here
cls_train_dataset = train_dataset.take(5000)

test_dataset = # here
test_dataset = test_dataset.map(prepare_ds)

In [ ]:
num_train_examples, num_valid_examples, num_test_examples = ds.dataset_split
num_train_steps = num_train_examples // batch_size
num_test_steps = num_test_examples // batch_size
num_train_examples, num_valid_examples, num_test_examples

In [ ]:
test_dataset

In [ ]:
first_image_aug_fn = iic_datasets_ops.get_default_image_aug_fn(rotation=0.1, skew=0.05, translation=0.1, crop=0.2)
second_image_aug_fn = iic_datasets_ops.get_default_image_aug_fn(crop=0.2)

test_image_aug_fn = iic_datasets_ops.get_default_image_aug_fn(
    rotation=0.0, skew=0.0, translation=0.0, crop=0.3, hue=0.0, saturation=0.0, brightness=0.0, uniform_noise=0.0, contrast=0.0
)


iic_train_dataset, model_input_shape = iic_datasets_ops.prepare_dataset(
    dataset=train_dataset,
    first_image_aug_fn=first_image_aug_fn,
    second_image_aug_fn=second_image_aug_fn,
    batch_size=batch_size,
    num_image_repeats=num_repeats,
    input_size=input_shape[:2],
)

iic_cls_train_dataset, _ = iic_datasets_ops.prepare_dataset(
    dataset=cls_train_dataset,
    first_image_aug_fn=first_image_aug_fn,
    second_image_aug_fn=second_image_aug_fn,
    batch_size=batch_size,
    num_image_repeats=num_repeats,
    input_size=input_shape[:2],
)

In [ ]:
iic_test_dataset, _ = iic_datasets_ops.prepare_dataset(
    dataset=test_dataset,
    first_image_aug_fn=test_image_aug_fn,
    second_image_aug_fn=test_image_aug_fn,
    batch_size=batch_size,
    num_image_repeats=num_repeats,
    input_size=input_shape[:2],
)

In [ ]:
iic_train_dataset, iic_test_dataset

In [ ]:
model_input_shape

In [ ]:
iic_train_iterator = iic_train_dataset.make_one_shot_iterator()
iic_test_iterator = iic_test_dataset.make_one_shot_iterator()
iic_cls_train_iterator = iic_cls_train_dataset.make_one_shot_iterator()
iic_train_iterator, iic_test_iterator

In [ ]:
utils.plot_image_pairs(*next(iic_train_iterator), num_examples=10)

In [ ]:
utils.plot_image_pairs(*next(iic_test_iterator), num_examples=10)

In [ ]:
utils.plot_image_pairs(*next(iic_cls_train_iterator), num_examples=10)

In [ ]:
%timeit -n 10 next(iic_train_iterator)

In [ ]:
base_model = models.create_resnet_se_backbone(model_input_shape, (4, 4, 4))

In [ ]:
base_model.summary()

In [ ]:
iic_prediction_model = models.create_iic_model(
    base_model, 
    main_heads_num_classes=[num_classes], 
    aux_heads_num_classes=[5 * num_classes],
)

In [ ]:
iic_prediction_model.summary()

In [ ]:
# inputs, main_heads, aux_heads
image_input = keras.Input(shape=model_input_shape, name="image")
tf_image_input = keras.Input(shape=model_input_shape, name="tf_image")

image_predictions = iic_prediction_model(image_input)
tf_image_predictions = iic_prediction_model(tf_image_input)

In [ ]:
image_predictions

In [ ]:
num_main_heads = len(image_predictions['main_heads'])
num_aux_heads = len(image_predictions['aux_heads'])

heads_p_out = image_predictions['main_heads']
heads_p_tf_out = tf_image_predictions['main_heads']

main_heads_loss = tf.add_n([
    iic_loss_ops.iic_loss(
        heads_p_out[k], 
        heads_p_tf_out[k])
    for k in range(num_main_heads)
]) / num_main_heads

aux_heads_loss = tf.add_n([
    iic_loss_ops.iic_loss(
        image_predictions['aux_heads'][k], 
        tf_image_predictions['aux_heads'][k]) 
    for k in range(num_aux_heads)
]) / num_aux_heads

main_heads_loss, aux_heads_loss

In [ ]:
inputs = {
    "image": image_input,
    "tf_image": tf_image_input
}

iic_main_model = keras.Model(inputs=inputs, outputs=heads_p_out)
iic_aux_model = keras.Model(inputs=inputs, outputs=heads_p_tf_out)

In [ ]:
iic_main_model.layers[-1].get_layer("aux_head_0/dense").trainable = False

In [ ]:
iic_main_model.add_loss(main_heads_loss)
iic_aux_model.add_loss(aux_heads_loss)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0005)
iic_main_model.compile(optimizer=optimizer)

optimizer = keras.optimizers.Adam(learning_rate=0.0005)
iic_aux_model.compile(optimizer=optimizer)

In [ ]:
iic_main_model.fit(
    iic_train_iterator, verbose=1, epochs=1,
    steps_per_epoch=100, 
    callbacks=[utils.PredictionsHistory(iic_test_iterator, num_classes=num_classes)]
)

In [ ]:
# utils.plot_probabilities_grid(iic_main_model, iic_test_iterator)

In [ ]:
labels = tf.keras.layers.Lambda(lambda x: x, name="label")(image_predictions['main_heads'][0])
cls_model = keras.Model(inputs={"image": image_input}, outputs=labels)
optimizer = keras.optimizers.Adam(learning_rate=0.0005)
cls_model.compile(optimizer=optimizer, loss={"label": "sparse_categorical_crossentropy"})

In [ ]:
cls_model.fit(
    iic_cls_train_dataset, verbose=1, epochs=1,
    steps_per_epoch=num_test_steps, callbacks = [utils.PredictionsHistory(iic_test_iterator, num_classes=num_classes)]
)

In [ ]:
def remap_dataset(pair_data, cls_data):
    pair_data[0]['cls_image'] = cls_data[0]['image']
    return {
        "image": pair_data[0]['image'],
        "tf_image": pair_data[0]['tf_image'],
        "cls_image": cls_data[0]['image']
    }, cls_data[1]

combined_dataset = tf.data.Dataset.zip((iic_train_dataset, iic_cls_train_dataset)).map(remap_dataset)
combined_dataset

In [ ]:
# inputs, main_heads, aux_heads
image_input = keras.Input(shape=model_input_shape, name="image")
tf_image_input = keras.Input(shape=model_input_shape, name="tf_image")
cls_image_input = keras.Input(shape=model_input_shape, name="cls_image")

image_predictions = iic_prediction_model(image_input)
tf_image_predictions = iic_prediction_model(tf_image_input)
cls_image_predictions = iic_prediction_model(cls_image_input)

In [ ]:
labels = tf.keras.layers.Lambda(lambda x: x, name="label")(cls_image_predictions['main_heads'][0])

In [ ]:
num_main_heads = len(image_predictions['main_heads'])
num_aux_heads = len(image_predictions['aux_heads'])

heads_p_out = image_predictions['main_heads']
heads_p_tf_out = tf_image_predictions['main_heads']

main_heads_loss = tf.add_n([
    iic_loss_ops.iic_loss(
        heads_p_out[k], 
        heads_p_tf_out[k])
    for k in range(num_main_heads)
]) / num_main_heads

aux_heads_loss = tf.add_n([
    iic_loss_ops.iic_loss(
        image_predictions['aux_heads'][k], 
        tf_image_predictions['aux_heads'][k]) 
    for k in range(num_aux_heads)
]) / num_aux_heads

main_heads_loss, aux_heads_loss

In [ ]:
cls_predictions

In [ ]:
inputs = {
    "image": image_input,
    "tf_image": tf_image_input,
    "cls_image": cls_image_input
}
iic_cls_model = keras.Model(inputs=inputs, outputs=labels)

In [ ]:
test_cls_model = keras.Model(inputs={"image": image_input}, outputs=heads_p_out)
test_cls_model.summary()

In [ ]:
iic_cls_model.layers[-4].get_layer("aux_head_0/dense").trainable = False

In [ ]:
iic_cls_model.add_loss(main_heads_loss)

In [ ]:
iic_cls_model.summary()

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0002)
iic_cls_model.compile(optimizer=optimizer, loss={"label": "sparse_categorical_crossentropy"}, metrics={"label": "accuracy"})

In [ ]:
iic_cls_model.fit(
    combined_dataset, verbose=1, epochs=15,
    steps_per_epoch=300, 
    callbacks=[
        utils.PredictionsHistory("test", test_cls_model, iic_test_iterator, num_classes=num_classes),
        utils.PredictionsHistory("train", test_cls_model, iic_train_iterator, num_classes=num_classes)
    ]
)

In [ ]:
test_cls_model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
def test_generator():
    while True:
        f, l = iic_test_iterator.get_next()
        yield f['image'][::2], l['label'][::2]

In [ ]:
test_cls_model.evaluate(test_generator(), steps=100)

In [ ]:
test_labels = []
for i in range(100):
    test_labels += iic_test_iterator.get_next()[1]['label'].numpy()[::2].tolist()

In [ ]:
from collections import Counter
len(Counter(test_labels)) / num_classes

In [ ]:
cls_image_predictions['main_heads'][0]

In [ ]:
image_input = keras.Input(shape=model_input_shape, name="image")
image_predictions = iic_prediction_model(image_input)

cls_labels = tf.keras.layers.Lambda(lambda x: x, name="label")(image_predictions['main_heads'][0])
cls_model_no_unsupervised = keras.Model(inputs={"image": image_input}, outputs=cls_labels)

In [ ]:
cls_model_no_unsupervised.layers[-2].get_layer("aux_head_0/dense").trainable = False

In [ ]:
cls_model_no_unsupervised.summary()

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0002)
cls_model_no_unsupervised.compile(
    optimizer=optimizer, loss={"label": "sparse_categorical_crossentropy"}, metrics={"label": "accuracy"})

In [ ]:
cls_model_no_unsupervised.fit(
    iic_cls_train_dataset, verbose=1, epochs=15,
    steps_per_epoch=300, 
    callbacks=[
        utils.PredictionsHistory("test", cls_model_no_unsupervised, iic_test_iterator, num_classes=num_classes),
        utils.PredictionsHistory("train", cls_model_no_unsupervised, iic_cls_train_iterator, num_classes=num_classes)
    ]
)

In [ ]:
def test_generator():
    while True:
        f, l = iic_test_iterator.get_next()
        yield f['image'][::2], l['label'][::2]

In [ ]:
cls_model_no_unsupervised.evaluate_generator(test_generator(), steps=100)

In [ ]:
def train_generator():
    while True:
        f, l = iic_cls_train_iterator.get_next()
        yield f['image'][::2], l['label'][::2]

In [ ]:
cls_model_no_unsupervised.evaluate(train_generator(), steps=100)